# LIBRERIAS 

* Tener en cuenta bajadas de Procedimientos, Personas Armas, Divisas , vehiculos (secuestrados, ministerio), Narcotrafico general,Objetos y pegar en la carpeta bajadas de fecha del primero del 1 del mes en informar hasta fecha del informe 

### IMPORTACIÓN DE LIBREARIAS


In [ ]:
import pandas as pd
from  Parametros import * 
from  Funciones import * 
from openpyxl import load_workbook


### ACCEDER BASE DE DATOS

In [ ]:
# df_base_operaciones = pd.read_excel("db/Base_operaciones_2023_2024.xlsx")
df_base_procedimientos = filtrar_procedimientos_generales("data/BASE_2023.xls")


# print(df_base_procedimientos.info())
# print(df_base_procedimientos.describe())
print(df_base_procedimientos.nunique())

# SIGIPOL

### PROCEDIMIENTOS GENERALES




In [ ]:
excel_bajada_procedimientos_generales = filtrar_procedimientos_generales("bajadas/bajada_general.xls")

df_geog = pd.DataFrame()
df_geog['UID'] = excel_bajada_procedimientos_generales.apply(generar_uid_sigpol, axis=1)
df_geog['ID_OPERATIVO'] = excel_bajada_procedimientos_generales.apply(procesar_causa_judicial, axis=1)
df_geog['FUERZA_INTERVINIENTE'] = "PSA"
df_geog['ID_PROCEDIMIENTO'] = df_geog['UID']
df_geog['CAUSAJUDICIALNUMERO'] = excel_bajada_procedimientos_generales['CAUSAJUDICIALNUMERO'].copy()

df_geog['UNIDAD_INTERVINIENTE'] = excel_bajada_procedimientos_generales['UOSP'].fillna(excel_bajada_procedimientos_generales['URSA'])

df_geog['DESCRIPCIÓN'] = excel_bajada_procedimientos_generales.apply(procesar_descripcion, axis=1)
df_geog['TIPO_INTERVENCION'] = excel_bajada_procedimientos_generales.apply(procesar_tipo_procedimiento, axis=1)
df_geog['PROVINCIA'] = excel_bajada_procedimientos_generales.apply(procesar_provincia, axis=1)
df_geog['DEPARTAMENTO O PARTIDO'] = excel_bajada_procedimientos_generales.apply(procesar_municipio, axis=1)
df_geog['LOCALIDAD'] = "-"
df_geog['DIRECCION'] = excel_bajada_procedimientos_generales.apply(procesar_direccion, axis=1)


df_geog[['LATITUD', 'LONGITUD']] = excel_bajada_procedimientos_generales.apply(procesar_geog, axis=1, result_type='expand')
df_geog['FECHA'] = pd.to_datetime(excel_bajada_procedimientos_generales['DENUNCIAFECHA'], errors='coerce').dt.strftime('%d/%m/%Y')
df_geog['HORA'] = pd.to_datetime(excel_bajada_procedimientos_generales['DENUNCIAFECHA'], errors='coerce').dt.strftime('%H:%M')
df_geog['ZONA_SEGURIDAD_FRONTERAS'] = "-"
df_geog['PASO_FRONTERIZO'] = "-"
df_geog['OTRAS AGENCIAS INTERVINIENTES'] = "-"
df_geog['Observaciones - Detalles'] = "-"


df_geog['LATITUD'] = df_geog['LATITUD'].astype(str).str.replace(',', '.')
df_geog['LONGITUD'] = df_geog['LONGITUD'].astype(str).str.replace(',', '.')

df_geog_completado = df_geog[[ "UID",'FUERZA_INTERVINIENTE', 'ID_OPERATIVO', 'ID_PROCEDIMIENTO',
                                     'UNIDAD_INTERVINIENTE', 'DESCRIPCIÓN', 'TIPO_INTERVENCION',
                                     'PROVINCIA', 'DEPARTAMENTO O PARTIDO', 'LOCALIDAD', 'DIRECCION',
                                     'ZONA_SEGURIDAD_FRONTERAS', 'PASO_FRONTERIZO', 'LATITUD', 'LONGITUD',
                                     'FECHA', 'HORA', 'OTRAS AGENCIAS INTERVINIENTES', 'Observaciones - Detalles']]

print(df_geog_completado.nunique())




### FILTRO CON LA BASE DE DATOS DE LO QUE SE FUE INFORMADO




In [ ]:
df_geog_final = df_geog_completado[~df_geog_completado['UID'].isin(set(df_base_procedimientos['UID']))]
print(f"\nCasos ya informados: {len(df_base_procedimientos)}")
print(f"Casos nuevos para informar : {len(df_geog_final)}")


print(df_geog_final.nunique())

### HOJA PERSONAS

In [ ]:

# Paso 1: Cargar datos iniciales
excel_bajada_personas = pd.read_excel("bajadas/bajada_general_persona.xls")
print(f"Total de registros cargados: {len(excel_bajada_personas)}")

# Paso 2: Generar UID
excel_bajada_personas['UOSP'] = excel_bajada_personas['UOSP'].fillna(excel_bajada_personas['URSA'])
excel_bajada_personas['TIPO_CAUSA_INTERNA'] = excel_bajada_personas.apply( procesar_tipo_causa_interna, axis=1)
excel_bajada_personas['UID'] = excel_bajada_personas.apply(generar_uid_sigpol, axis=1)

# Paso 3: Coincidencia de UID entre excel_bajada_personas y df_geog_final
uid_coincidentes = excel_bajada_personas[excel_bajada_personas['UID'].isin(df_geog_final['UID'])]
print(f"Cantidad de registros con UID coincidente: {len(uid_coincidentes)}")

# Crear DataFrame de detenidos y aprendidos
df_detenidos_aprendidos = pd.DataFrame()
df_detenidos_aprendidos['UID'] = uid_coincidentes['UID']
df_detenidos_aprendidos['EDAD'] = uid_coincidentes.apply(procesar_edad, axis=1)
df_detenidos_aprendidos['SEXO'] = uid_coincidentes.apply(procesar_sexo, axis=1)
df_detenidos_aprendidos['GENERO'] = uid_coincidentes.apply(procesar_genero, axis=1)
df_detenidos_aprendidos['NACIONALIDAD'] = uid_coincidentes.apply(procesar_nacionalidad, axis=1)
df_detenidos_aprendidos['SITUACION_PROCESAL'] = uid_coincidentes.apply(procesar_situacion_judicial, axis=1)
df_detenidos_aprendidos['DELITO_IMPUTADO'] = uid_coincidentes.apply(procesar_tipo_delito, axis=1)
df_detenidos_aprendidos['JUZGADO_INTERVINIENTE'] = uid_coincidentes.apply(procesar_juzgado, axis=1)
df_detenidos_aprendidos['CARATULA_CAUSA'] = uid_coincidentes.apply(procesar_caratula, axis=1)

print(f"Total de registros después del procesamiento de columnas específicas: {len(df_detenidos_aprendidos)}")

# Paso 4: Filtrar SITUACION_PROCESAL 'NO INFORMAR'
df_detenidos_aprendidos = df_detenidos_aprendidos[df_detenidos_aprendidos['SITUACION_PROCESAL'] != 'NO INFORMAR']
print(f"Cantidad tras exclusión de 'NO INFORMAR': {len(df_detenidos_aprendidos)}")

# Paso 5: Fusionar df_geog_final y df_detenidos_aprendidos en base al UID
df_detenidos_aprendidos = pd.merge(df_geog_final, df_detenidos_aprendidos, on='UID', how='left')
print(f"Total de registros después de la fusión con df_geog_final: {len(df_detenidos_aprendidos)}")

# Paso 6: Relleno de valores vacíos con '-'
df_detenidos_aprendidos = df_detenidos_aprendidos.apply(lambda x: x.fillna('-') if x.dtype == 'object' else x)


# Paso 7: Selección y reordenamiento de columnas
df_detenidos_aprendidos = df_detenidos_aprendidos[[
    "UID",
    'FUERZA_INTERVINIENTE',
    'ID_OPERATIVO',
    'ID_PROCEDIMIENTO',
    'UNIDAD_INTERVINIENTE',
    'DESCRIPCIÓN',
    'TIPO_INTERVENCION',
    'EDAD',
    'SEXO',
    'GENERO',
    'NACIONALIDAD',
    'SITUACION_PROCESAL',
    "DELITO_IMPUTADO",
    'JUZGADO_INTERVINIENTE',
    'CARATULA_CAUSA'
]]

print(f"Total de registros después de seleccionar y reordenar columnas: {len(df_detenidos_aprendidos)}")


### HOJA OTROS DELITOS

In [ ]:
# Crear un nuevo dataframe para otro delitos
df_otros_delitos = df_detenidos_aprendidos[df_detenidos_aprendidos['SITUACION_PROCESAL'] == 'VICTIMA']
# Filtro los que son victima de detenidos y aprendidos
df_detenidos_aprendidos_final = df_detenidos_aprendidos[df_detenidos_aprendidos['SITUACION_PROCESAL'] != 'VICTIMA']

# Cambiar el nombre de algunas columnas en el DataFrame de víctimas
df_otros_delitos = df_otros_delitos.rename(columns={
    'DELITO_IMPUTADO': 'TIPO_OTRO_DELITO',
    'GENERO': 'GENERO_VICTIMA',
    'EDAD': 'EDAD_VICTIMA',
}) # type: ignore

df_otros_delitos["OBSERVACIONES"] ="-"
# Seleccionar las columnas deseadas en el DataFrame de víctimas
df_otros_delitos_final = df_otros_delitos[[
    "UID",
    'FUERZA_INTERVINIENTE',
    'ID_OPERATIVO',
    'ID_PROCEDIMIENTO',
    'UNIDAD_INTERVINIENTE',
    'DESCRIPCIÓN',
    'TIPO_INTERVENCION',
    "TIPO_OTRO_DELITO",
    'GENERO_VICTIMA',
    'EDAD_VICTIMA',
    'NACIONALIDAD',
    'OBSERVACIONES'
]]


# Imprimir la cantidad de registros en el DataFrame de otros delitos para verificar
print(f"Cantidad de registros en geog: {len(df_geog_final)}")
print(f"Cantidad de registros en detenidos_aprendidos: {len(df_detenidos_aprendidos_final)}")
print(f"Cantidad de registros en otros_delitos: {len(df_otros_delitos_final)}")

### INCAUTACIONES ARMA

In [ ]:
# Paso 1: Cargar datos iniciales
excel_bajada_arma = pd.read_excel("bajadas/bajada_arma.xls")
print(f"Total de registros cargados: {len(excel_bajada_arma)}")

# Paso 2: Generar UID
excel_bajada_arma = excel_bajada_arma[excel_bajada_arma["TIPO_ESTADO_OBJETO"] == "SECUESTRADO"]
excel_bajada_arma['UOSP'] = excel_bajada_arma['UOSP'].fillna(excel_bajada_arma['URSA'])
excel_bajada_arma['TIPO_CAUSA_INTERNA'] = excel_bajada_arma.apply( procesar_tipo_causa_interna, axis=1)
excel_bajada_arma['UID'] = excel_bajada_arma.apply(generar_uid_sigpol, axis=1)


# Crear DataFrame de arma
df_arma = pd.DataFrame()
df_arma['UID'] = excel_bajada_arma['UID']
df_arma['TIPO_INCAUTACION'] = "ARMAS"
df_arma['TIPO'] = excel_bajada_arma['TIPO_ARMA']
df_arma['SUBTIPO'] = "-"
df_arma['CANTIDAD'] = excel_bajada_arma.apply(procesar_cantidad_arma, axis=1)
df_arma['MEDIDAS'] = "UNIDADES"
df_arma['AFORO'] = "-"
df_arma['OBSERVACIONES'] = excel_bajada_arma.apply(procesar_observaciones_arma, axis=1)

# Paso 3: Coincidencia de UID entre excel_bajada_arma y df_geog_final
df_arma_final = df_arma[df_arma['UID'].isin(df_geog_final['UID'])]
print(df_arma_final)


### INCAUTACIONES DIVISA

In [ ]:
# Paso 1: Cargar datos iniciales
excel_bajada_divisa = pd.read_excel("bajadas/bajada_divisas_secuestradas.xls")
print(f"Total de registros cargados: {len(excel_bajada_divisa)}")

excel_bajada_procedimientos_generales['PARTE_ANIO'] = excel_bajada_procedimientos_generales['NUMERO_PARTE'].astype(str) + "/" + excel_bajada_procedimientos_generales['ANIO_PARTE'].astype(str)
excel_bajada_divisa['PARTE_ANIO'] = excel_bajada_divisa['NUMERO_PARTE'].astype(str) + "/" + excel_bajada_divisa['ANIO_PARTE'].astype(str)

excel_bajada_divisa['UOSP'] = excel_bajada_divisa['UOSP'].fillna(excel_bajada_divisa['URSA'])
# Hacer el merge para traer la columna TIPO_CAUSA_INTERNA a excel_bajada_divisa
excel_bajada_divisa = pd.merge(excel_bajada_divisa, 
                               excel_bajada_procedimientos_generales[['PARTE_ANIO', 'TIPO_CAUSA_INTERNA']], 
                               on='PARTE_ANIO', 
                               how='left')

excel_bajada_divisa['UID'] = excel_bajada_divisa.apply(generar_uid_sigpol, axis=1)
excel_bajada_divisa = excel_bajada_divisa[~pd.isnull(excel_bajada_divisa['TOTAL_DIVISAS_SECUESTRADAS'])]


# Agrupar por 'UID' (y 'TIPO' si se necesita) para sumar las cantidades
cantidad_por_uid = excel_bajada_divisa.groupby(['UID', 'TIPO_DIVISA'], as_index=False)['TOTAL_DIVISAS_SECUESTRADAS'].sum()

# Paso 2: Crear df_divisa con las cantidades sumadas
df_divisa = pd.DataFrame()
df_divisa['UID'] = cantidad_por_uid['UID']
df_divisa['TIPO_INCAUTACION'] = "DIVISAS"
df_divisa['TIPO'] = cantidad_por_uid['TIPO_DIVISA']
df_divisa['SUBTIPO'] = "-"
df_divisa['CANTIDAD'] = cantidad_por_uid['TOTAL_DIVISAS_SECUESTRADAS']
df_divisa['MEDIDAS'] = "UNIDADES"
df_divisa['AFORO'] = "-"
df_divisa['OBSERVACIONES'] = "-"

# Mostrar el resultado final
df_divisa_final = df_divisa[df_divisa['UID'].isin(df_geog_final['UID'])]


print(df_divisa_final.nunique())
print(df_divisa_final)

### INCAUTACIONES OBJETOS

In [ ]:
# Paso 1: Cargar datos iniciales
excel_bajada_objetos = pd.read_excel("bajadas/bajada_objetos.xls")
print(f"Total de registros cargados: {len(excel_bajada_objetos)}")

excel_bajada_objetos = excel_bajada_objetos[excel_bajada_objetos["ESTADO"] == "SECUESTRADO"]

excel_bajada_objetos['UOSP'] = excel_bajada_objetos['UOSP'].fillna(excel_bajada_objetos['URSA'])
excel_bajada_objetos['TIPO_CAUSA_INTERNA'] = excel_bajada_objetos.apply( procesar_tipo_causa_interna, axis=1)
excel_bajada_objetos['UID'] = excel_bajada_objetos.apply(generar_uid_sigpol, axis=1)
excel_bajada_objetos = excel_bajada_objetos[excel_bajada_objetos['TIPO_OBJETO'].isin(TIPO_OBJETO)]
excel_bajada_objetos = excel_bajada_objetos[~pd.isnull(excel_bajada_objetos['CANTIDAD'])]

# Paso 2: Crear df_divisa con las cantidades sumadas
df_objetos = pd.DataFrame()
df_objetos['UID'] = excel_bajada_objetos['UID']
df_objetos['TIPO_INCAUTACION'] = "MERCADERIA"
df_objetos['TIPO'] = excel_bajada_objetos['TIPO_OBJETO']
df_objetos['SUBTIPO'] = "-"
df_objetos['CANTIDAD'] = excel_bajada_objetos['CANTIDAD']
df_objetos['MEDIDAS'] = "UNIDADES"
df_objetos['AFORO'] = "-"
df_objetos['OBSERVACIONES'] = "-"

# Mostrar el resultado final


df_objetos_final = df_objetos[df_objetos['UID'].isin(df_geog_final['UID'])]

print(df_objetos_final.nunique())
print(df_objetos_final)

### INCAUTACIONES VEHICULOS

In [ ]:
# Paso 1: Cargar datos iniciales
excel_bajada_vehiculos = pd.read_excel("bajadas/bajada_vehiculos_secuestrados.xls")
excel_bajada_ministerio_vehiculos = pd.read_excel("bajadas/bajada_ministerio_vehiculos.xls")
print(f"Total de registros vehiculos secuestrados: {len(excel_bajada_vehiculos)}")
print(f"Total de registros ministerio vehiculos secuestrados: {len(excel_bajada_ministerio_vehiculos)}")


excel_bajada_vehiculos = excel_bajada_vehiculos[excel_bajada_vehiculos["VEHICULO_ESTADO"] == "SECUESTRADO"]

excel_bajada_ministerio_vehiculos['TIPO_CAUSA_INTERNA'] = excel_bajada_vehiculos.apply( procesar_tipo_causa_interna, axis=1)
excel_bajada_vehiculos['UOSP'] = excel_bajada_vehiculos['UOSP'].fillna(excel_bajada_vehiculos['URSA'])
excel_bajada_vehiculos = pd.merge(
    excel_bajada_vehiculos, 
    excel_bajada_ministerio_vehiculos[[  'NUMERO_PARTE', 'ANIO_PARTE', 'VEHICULO_DOMINIO']], 
    on='VEHICULO_DOMINIO', 
    how='left'
)
excel_bajada_vehiculos['TIPO_CAUSA_INTERNA'] = excel_bajada_vehiculos.apply( procesar_tipo_causa_interna, axis=1)
excel_bajada_vehiculos['UID'] = excel_bajada_vehiculos.apply(generar_uid_sigpol, axis=1)


# Paso 2: Crear df_divisa con las cantidades sumadas
df_vehiculos = pd.DataFrame()
df_vehiculos['UID'] = excel_bajada_vehiculos['UID']
df_vehiculos['TIPO_INCAUTACION'] = "SECUESTRO DE VEHICULOS"
# df_vehiculos['TIPO'] = excel_bajada_vehiculos.apply(clasificar_tipo_vehiculo, axis=1)
df_vehiculos['TIPO'] = excel_bajada_vehiculos['VEHICULO_TIPO']
df_vehiculos['SUBTIPO'] = "-"
df_vehiculos['CANTIDAD'] = excel_bajada_vehiculos['CANTIDAD']
df_vehiculos['MEDIDAS'] = "UNIDADES"
df_vehiculos['AFORO'] = "-"
df_vehiculos['OBSERVACIONES'] = excel_bajada_vehiculos.apply(observaciones_vehiculo,axis=1)


df_vehiculos_final = df_vehiculos[df_vehiculos['UID'].isin(df_geog_final['UID'])]
print(f"Cantidad de registros con UID coincidente: {len(uid_coincidentes)}")

print(df_vehiculos_final.nunique())
print(df_vehiculos_final)

### INCAUTACIONES NARCOTRAFICO

In [ ]:
# # Paso 1: Cargar datos iniciales
# excel_bajada_narcotrafico = pd.read_excel("bajadas/bajada_general_narcotrafico.xls")
# print(f"Total de registros narcostrafico secuestrados: {len(excel_bajada_narcotrafico)}")

# excel_bajada_narcotrafico['UOSP'] = excel_bajada_narcotrafico['UOSP'].fillna(excel_bajada_narcotrafico['URSA'])
# excel_bajada_narcotrafico['TIPO_CAUSA_INTERNA'] = excel_bajada_narcotrafico.apply( procesar_tipo_causa_interna, axis=1)
# excel_bajada_narcotrafico['UID'] = excel_bajada_narcotrafico.apply(generar_uid_sigpol, axis=1)

# excel_bajada_narcotrafico = excel_bajada_narcotrafico[~pd.isnull(excel_bajada_narcotrafico['TIPO_ESTUPEFACIENTE'])]


# #Paso 2: Crear df_divisa con las cantidades sumadas
# df_narcotrafico = pd.DataFrame()
# df_narcotrafico['UID'] = excel_bajada_narcotrafico['UID']
# df_narcotrafico['TIPO_INCAUTACION'] = "ESTUPEFACIENTE"
# df_narcotrafico['TIPO'] = excel_bajada_narcotrafico.apply(clasificar_tipo_sustancia, axis=1)
# df_narcotrafico['SUBTIPO'] = "-"
# df_narcotrafico[['CANTIDAD', 'MEDIDAS']] = excel_bajada_narcotrafico.apply(clasificar_medida, axis=1, result_type='expand')
# df_narcotrafico['AFORO'] = "-"
# df_narcotrafico['OBSERVACIONES'] = excel_bajada_narcotrafico.apply(observaciones_sustancia, axis=1)



### UNION DE ARMAS, DIVISAS, OBJETOS , VEHICULOS , NARCOTRAFICO

In [ ]:
# df_incautados = pd.concat([df_objetos, df_vehiculos, df_narcotrafico, df_arma, df_divisa])
# df_incautados = pd.concat([df_objetos, df_vehiculos, df_arma, df_divisa])
df_incautados = pd.concat([df_objetos_final, df_vehiculos_final, df_arma_final, df_divisa_final])
df_incautados = df_incautados.apply(lambda x: x.fillna('-') if x.dtype == 'object' else x)


print(f"Total de registros incautados sin filtracion de lo que ya se informo: {len(df_incautados)}")


df_incautados_filtrado = df_incautados[df_incautados['UID'].isin(df_geog_final['UID'])]


print(f"Total de registros incautados con filtracion de lo que ya se informo: {len(df_incautados_filtrado)}")

df_incautados_completado = pd.merge(df_geog_final, df_incautados, on='UID', how='right')
# Paso 6: Relleno de valores vacíos con '-'


# Paso 7: Selección y reordenamiento de columnas
df_incautados_final = df_incautados_completado[[
    "UID",
    'FUERZA_INTERVINIENTE',
    'ID_OPERATIVO',
    'ID_PROCEDIMIENTO',
    'UNIDAD_INTERVINIENTE',
    'DESCRIPCIÓN',
    'TIPO_INTERVENCION',
    'TIPO_INCAUTACION',
    'TIPO',
    'SUBTIPO',
    'CANTIDAD',
    'MEDIDAS',
    "AFORO",
    'OBSERVACIONES'
]]


print(df_incautados_final.nunique())
print(df_incautados_final)

### TRATA DE PERSONA

In [ ]:
excel_bajada_trata = pd.read_excel("bajadas/bajada_trata.xls")
print(f"Total de registros tratas: {len(excel_bajada_trata)}")

excel_bajada_trata['UOSP'] = excel_bajada_trata['UOSP'].fillna(excel_bajada_trata['URSA'])
excel_bajada_trata['TIPO_CAUSA_INTERNA'] = excel_bajada_trata.apply( procesar_tipo_causa_interna, axis=1)
excel_bajada_trata['UID'] = excel_bajada_trata.apply(generar_uid_sigpol, axis=1)

# Crear DataFrame de detenidos y aprendidos
df_trata = pd.DataFrame()
df_trata['UID'] = excel_bajada_trata['UID']
df_trata['TIPO_DELITO'] = excel_bajada_trata["TIPO_EXPLOTACION"].fillna("-")
df_trata['SEXO_VICTIMA'] = excel_bajada_trata.apply(procesar_sexo, axis=1)
df_trata['GENERO_VICTIMA'] = excel_bajada_trata.apply(procesar_genero, axis=1)
df_trata['EDAD_VICTIMA'] = excel_bajada_trata['EDAD'].fillna("-")
df_trata['NACIONALIDAD'] = excel_bajada_trata['NACIONALIDAD'].fillna("-")
df_trata['NACIONALIDAD'] = excel_bajada_trata['NACIONALIDAD'].str.upper()
df_trata['JUZGADO_INTERVINIENTE'] = excel_bajada_trata.apply(procesar_juzgado, axis=1)
df_trata['OBSERVACIONES'] = "-"

df_trata_filtrado = df_trata[df_trata['UID'].isin(df_geog_final['UID'])]

df_trata_completado = pd.merge(df_geog_final, df_trata_filtrado, on='UID', how='right')


df_trata_final = df_trata_completado[[
    "UID",
    'FUERZA_INTERVINIENTE',
    'ID_OPERATIVO',
    'ID_PROCEDIMIENTO',
    'UNIDAD_INTERVINIENTE',
    'DESCRIPCIÓN',
    'TIPO_INTERVENCION',
    "TIPO_DELITO",
    'SEXO_VICTIMA',
    'GENERO_VICTIMA',
    'EDAD_VICTIMA',
    'NACIONALIDAD',
    'JUZGADO_INTERVINIENTE',
    'OBSERVACIONES'
]]


print(df_trata_final.nunique())
print(df_trata_final)

# OPERACIONES

### ORDEN DE SERVICIOS

In [ ]:
excel_bajada_operaciones = pd.read_excel("bajadas/PLANILLA DE CARGA OPERACIONES.xlsm", 
                                         sheet_name="ORDEN_SERVICIOS", 
                                         skiprows=1)


df_operaciones= pd.DataFrame()
df_operaciones["UID"] = excel_bajada_operaciones["ID_PROCEDIMIENTO"]
df_operaciones["FUERZA_INTERVINIENTE"] = excel_bajada_operaciones["FUERZA_INTERVINIENTE"]
df_operaciones["ID_PROCEDIMIENTO"] = excel_bajada_operaciones["ID_PROCEDIMIENTO"]
df_operaciones["ID_OPERATIVO"] = excel_bajada_operaciones["ID_OPERATIVO"]
df_operaciones["UNIDAD_INTERVINIENTE"] = excel_bajada_operaciones["UNIDAD_INTERVINIENTE"]
df_operaciones["DESCRIPCION"] = excel_bajada_operaciones["DESCRIPCION"]
df_operaciones["TIPO_INTERVENCION"] = excel_bajada_operaciones["TIPO_INTERVENCION"]
df_operaciones["PROVINCIA"] = excel_bajada_operaciones["PROVINCIA"]
df_operaciones["DEPARTAMENTO O PARTIDO"] = excel_bajada_operaciones["DEPARTAMENTO O PARTIDO"]
df_operaciones["LOCALIDAD"] = excel_bajada_operaciones["LOCALIDAD"]
df_operaciones["DIRECCION"] = excel_bajada_operaciones["DIRECCION"]
df_operaciones["ZONA_SEGURIDAD_FRONTERAS"] = "-"
df_operaciones["PASO_FRONTERIZO"] = "-"
df_operaciones["LATITUD"] = excel_bajada_operaciones["LATITUD"]
df_operaciones["LONGITUD"] = excel_bajada_operaciones["LONGITUD"]
df_operaciones["FECHA"] = excel_bajada_operaciones["FECHA"]
df_operaciones["HORA"] = excel_bajada_operaciones["HORA"]



df_geog['DESCRIPCIÓN'] = excel_bajada_procedimientos_generales.apply(procesar_descripcion, axis=1)
df_geog['TIPO_INTERVENCION'] = excel_bajada_procedimientos_generales.apply(procesar_tipo_procedimiento, axis=1)
df_geog['PROVINCIA'] = excel_bajada_procedimientos_generales.apply(procesar_provincia, axis=1)
df_geog['DEPARTAMENTO O PARTIDO'] = excel_bajada_procedimientos_generales.apply(procesar_municipio, axis=1)
df_geog['LOCALIDAD'] = "-"
df_geog['DIRECCION'] = excel_bajada_procedimientos_generales.apply(procesar_direccion, axis=1)


df_geog[['LATITUD', 'LONGITUD']] = excel_bajada_procedimientos_generales.apply(procesar_geog, axis=1, result_type='expand')
df_geog['FECHA'] = pd.to_datetime(excel_bajada_procedimientos_generales['DENUNCIAFECHA'], errors='coerce').dt.strftime('%d/%m/%Y')
df_geog['HORA'] = pd.to_datetime(excel_bajada_procedimientos_generales['DENUNCIAFECHA'], errors='coerce').dt.strftime('%H:%M')
df_geog['ZONA_SEGURIDAD_FRONTERAS'] = "-"
df_geog['PASO_FRONTERIZO'] = "-"
df_geog['OTRAS AGENCIAS INTERVINIENTES'] = "-"
df_geog['Observaciones - Detalles'] = "-"
# df_operaciones = df_operaciones[[ "UID",'FUERZA_INTERVINIENTE', 'ID_OPERATIVO', 'ID_PROCEDIMIENTO',
#                                      'UNIDAD_INTERVINIENTE', 'DESCRIPCIÓN', 'TIPO_INTERVENCION',
#                                      'PROVINCIA', 'DEPARTAMENTO O PARTIDO', 'LOCALIDAD', 'DIRECCION',
#                                      'ZONA_SEGURIDAD_FRONTERAS', 'PASO_FRONTERIZO', 'LATITUD', 'LONGITUD',
#                                      'FECHA', 'HORA', 'OTRAS AGENCIAS INTERVINIENTES', 'Observaciones - Detalles']]


# print(f"Cantidad de registros en df_otros_delitos: {len(df_operaciones)}")

# Aplicar la función para identificar las filas que son diferentes

# df_operaciones = colocar_contador(df_operaciones , df_base_operaciones)




### GENERACIÓN DE INFORME

In [ ]:
from datetime import datetime
from openpyxl import load_workbook
from openpyxl.styles import Alignment


# Preparar los DataFrames
dataframes = {
    'GEOG. PROCEDIMIENTO': df_geog_final.drop(columns=['UID'], errors='ignore').reset_index(drop=True).fillna(""),
    'INCAUTACIONES': df_incautados_final.drop(columns=['UID'], errors='ignore').reset_index(drop=True).fillna(""),
    'DETENIDOS Y APREHENDIDOS': df_detenidos_aprendidos_final.drop(columns=['UID'], errors='ignore').reset_index(drop=True).fillna(""),
    'OTROS DELITOS': df_otros_delitos_final.drop(columns=['UID'], errors='ignore').reset_index(drop=True).fillna(""),
    'TRATA O TRAFIC PERSONAS': df_trata_final.drop(columns=['UID'], errors='ignore').reset_index(drop=True).fillna("")
}

# Cargar la plantilla
wb = load_workbook('Planilla_modelo.xlsx')

# Escribir los datos
for sheet_name, df in dataframes.items():
    sheet = wb[sheet_name]
    for i, row in df.iterrows():
        for j, value in enumerate(row):
            cell= sheet.cell(row=i+4, column=j+2, value=value) # type: ignore
            cell.alignment = Alignment(horizontal='center')
            


for sheet in wb.worksheets:
    sheet.conditional_formatting = []
    sheet.auto_filter.ref = None
    sheet.auto_filter = None
# Guardar el archivo
output_file = f'Informe_semanal_{datetime.now().strftime("%Y%m%d")}.xlsx'
wb.save(output_file)
print(f"Archivo generado: {output_file}")
